# Group Chat Orchestration - Round Robin Group Chat Manager

Group chat orchestration models a collaborative conversation among agents, optionally including a human participant. A group chat manager coordinates the flow, determining which agent should respond next and when to request human input. This pattern is powerful for simulating meetings, debates, or collaborative problem-solving sessions.

[Agent-orchestration Guide - Group Chat Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/group-chat?pivots=programming-language-python)
* [Original Code](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started_with_agents/multi_agent_orchestration/step3_group_chat.py)

The following sample demonstrates how to create a group chat orchestration with a default
round robin manager for controlling the flow of conversation in a round robin fashion.

Think of the group chat manager as a state machine, with the following possible states:
- Request for user message
- Termination, after which the manager will try to filter a result from the conversation
- Continuation, at which the manager will select the next agent to speak

This sample demonstrates the basic steps of creating and starting a runtime, creating
a group chat orchestration with a group chat manager, invoking the orchestration,
and finally waiting for the results.

There are two agents in this orchestration: a writer and a reviewer. They work iteratively
to refine a slogan for a new electric SUV.

In [31]:
# Libraries
from dotenv import load_dotenv
import os
import asyncio

from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent

In [32]:
#load variables
load_dotenv()

# Variables - Azure Services
azopenai_ep=os.environ["AZURE_OPENAI_ACCOUNT"]
azopenai_key=os.environ["AZURE_OPENAI_KEY"]
azopenai_model=os.environ["AZURE_OPENAI_MODEL"]

### Create an instance of AzureChatCompletion with the provided configuration

In [33]:
az_chat_completion = AzureChatCompletion(
    deployment_name=azopenai_model,
    api_key=azopenai_key,
    endpoint=azopenai_ep 
)

### Create the agents

In [34]:
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the group style discussion.

    Feel free to add or remove agents.
    """
    writer = ChatCompletionAgent(
        name="Architect",
        description="An AI Technical Architect.",
        instructions=(
            "You are an experienced AI architect. You design Azure AI Solutions and the respective documentation."
        ),
        service=az_chat_completion
    )
    reviewer = ChatCompletionAgent(
        name="Reviewer",
        description="A Technical content reviewer.",
        instructions=(
            "You are an excellent content reviewer. You review the content and provide feedback to the Architect."
        ),
        service=az_chat_completion
    )

    # The order of the agents in the list will be the order in which they will be picked by the round robin manager
    return [writer, reviewer]

### Optional: Observe Agent Responses

You can define a callback to observe and print the output from each agent as the sequence progresses.

In [35]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    print(f"**{message.name}**\n{  message.content}\n\n")

### Set Up the Group Chat Orchestration

Create a GroupChatOrchestration object, passing in the agents, a group chat manager (in this case a RoundRobinGroupChatManager), and the response callback. The manager controls the flow—here, it alternates turns in a round-robin fashion for a set number of rounds.

In [36]:
# 1. Create a group chat orchestration with a round robin manager
agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    # max_rounds is odd, so that the writer gets the last round
    manager=RoundRobinGroupChatManager(max_rounds=5),
    agent_response_callback=agent_response_callback,
)

### Start the Runtime

Start the runtime to manage agent execution.

In [ ]:
# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

**Architect**
Understood. Adopting the role of Azure AI Architect. I will refine and structure the content accordingly.

---

### Final Document: Refined and Professional Version  

#### Key Responsibilities and Deliverables  

- **Understand Business Objectives**  
  Gain a deep understanding of the customer's business goals, core requirements, and key outcomes to align the architecture strategy with their organizational priorities.  

- **Evaluate Current Architecture**  
  Review the existing architecture principles, design constraints, and the current state of workloads to identify compatibility and enhancement opportunities.  

- **Conduct Workload Assessment**  
  Analyze the existing customer workloads to understand their functionalities, dependencies, and integration touchpoints with Azure AI services.  

- **Document High-Level Architecture**  
  Deliver a comprehensive architecture design document, outlining the proposed high-level solution that meets the customer’s objective

### Invoke the Orchestration

Invoke the orchestration with your initial task (e.g., "Create a slogan for a new electric SUV..."). The agents will take turns responding, refining the result.

In [38]:
orchestration_result = await group_chat_orchestration.invoke(    
    task="Do the following with the text inside triple hyphen: \
- Correct grammar if needed \
- Correct misspelling if needed \
- Merge duplicated bullet points \
- Remove unnecessary bullet points \
- Rewrite bullet points using professional wording and structure if needed \
--- \
- Understanding the customer’s business goals and key requirements. \
- Understanding current architecture principles and design boundaries. \
- Discover and understand existing customer workload \
- Deliver an architecture design document that covers high-level architecture proposal that the customer also agrees on. \
- Understand the current state and desired solution.​ \
- Understand any dependencies such as existing applications and interfaces, their current functionalities, and architecture, to identify necessary changes required for the integration of modernization ideas centered around Azure AI services. \
- Review the existing infrastructure, evaluating its capacity to support the integration of Azure AI services while ensuring reliability, scalability, and security. \
- Assess various components of the infrastructure, including compute resources, network architecture, and data storage solutions, to identify potential gaps or constraints. \
- Formulate actionable recommendations for enhancing resilience, scalability, and security to accommodate the changes needed to complete the modernization efforts. \
- Include effective monitoring and management practices for AI-powered applications into the architecture. Incorporate market leading Azure services to track performance metrics, detect anomalies, and troubleshoot issues proactively. \
- Envision future capabilities while defining what's possible with Azure's generative AI offerings today. \
---",
    runtime=runtime,
)


### Collect Results and Stop the Runtime

In [39]:
# 4. Wait for the results
value = await orchestration_result.get()
print(f"***** Result *****\n{value}")

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()

***** Result *****
Adopting the "Architect" persona immediately.

---

### Response to Review Feedback  

Thank you for the insightful feedback and for highlighting the opportunities to further strengthen the vision statement and infrastructure assessment sections. I will now incorporate the recommended refinements to enhance technical depth, specificity, and clarity while maintaining the overall professional tone.

---

### Final Updated Document: Enhanced with Review Suggestions

#### Key Responsibilities and Deliverables  

- **Understand Business Objectives**  
  Gain a deep understanding of the customer's business goals, core requirements, and key outcomes to align the architecture strategy with their organizational priorities.  

- **Evaluate Current Architecture**  
  Review the existing architecture principles, design constraints, and the current state of workloads to identify compatibility and enhancement opportunities.  

- **Conduct Workload Assessment**  
  Analyze the exis